In [97]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Embedding
import keras.backend as K

from sklearn.feature_extraction.text import CountVectorizer

import pickle as pkl
import pandas as pd
import numpy as np
import multiprocessing
import unicodedata

import gensim
import nltk
#nltk.download('word2vec_sample')
from nltk.data import find

In [2]:
# import train and test data
df_test = pkl.load(open('Train_Test_Data/genre_sub_genre_test.pkl', 'rb'))
df_train = pkl.load(open('Train_Test_Data/genre_sub_genre_train.pkl', 'rb'))

In [104]:
print(len(df_train))
print(len(df_test))

16500
2875


In [99]:
def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')

In [ ]:
def strip_versions(text):
    text = text.split()
    if '-'
        

In [103]:
foo = ['a', 1]
print('a' in foo)

True


In [100]:
df_train['Artist Name'] = df_train['Artist Name'].map(lambda x: strip_accents(x))
df_train['Track Name'] = df_train['Track Name'].map(lambda x: strip_accents(x))

In [101]:
df_train

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,Sub-Genre: southern hip hop,Sub-Genre: nu metal,Sub-Genre: israeli mediterranean,Sub-Genre: thrash metal,Sub-Genre: pop rock,Sub-Genre: chicago blues,Sub-Genre: indie pop,Sub-Genre: classic rock,Sub-Genre: hardcore hip hop,normalized_audio_feature_array
16303,Escape the Fate,Unbreakable,47,0.563,0.820,1,-6.740,0,0.0489,0.00546,...,0,0,0,0,0,0,0,0,0,"[0.08667260380184501, 0.5543217870346929, 0.1220273049840551, -0.674109859950688, -0.41543317804459345, -0.440099973618899, 0.27353173919337254, -0.04503025561781331, -0.63357693290532]"
8721,COUCOU CHLOE,NOBODY,46,0.844,0.787,11,-7.044,1,0.1320,0.09410,...,0,0,0,0,0,0,0,0,0,"[1.769134455639869, 0.3955264896487572, 0.02601600085691811, -0.3318981833522548, 0.5337352175503375, -0.4389348974290065, 0.19657635700507578, -0.45322352624784257, -1.4860406055792426]"
11930,beabadoobee,Worth It,51,0.576,0.751,2,-5.256,1,0.0269,0.01250,...,0,0,0,0,0,0,1,0,0,"[0.16450891723207395, 0.22229525613682674, 0.5907140659204748, -0.6469305932172383, -0.6667172298266449, -0.4406102259648373, -0.48747148466867257, -0.7234102089486819, -0.4817594483694913]"
7945,Andrew Broder,Bloodrush,39,0.693,0.613,11,-10.160,0,0.2610,0.07980,...,0,0,0,0,0,0,0,0,0,"[0.8650357381041337, -0.4417578056589061, -0.9580998664462383, -0.3871060689045742, 2.007173521181458, -0.4419794030931051, -0.8551471995683124, 0.19473671280702892, -0.3577680344419983]"
15504,Greta Van Fleet,Age Of Man,56,0.460,0.601,6,-5.486,0,0.0304,0.03770,...,0,0,0,0,0,0,0,0,0,"[-0.5300304949145832, -0.49950155016288295, 0.5180739345084961, -0.5496411725236405, -0.6267402215885913, 2.597691620027714, -1.056086253059976, 0.42505256733097685, 1.7496968265126795]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12579,ABSOLUTE.,Sage comme une image - Good as Gold,44,0.792,0.873,6,-8.622,0,0.3290,0.00402,...,0,0,0,0,0,0,0,0,0,"[1.457789201918954, 0.8093566585939238, -0.4723584659609191, -0.6796692554188936, 2.7838696812350716, -0.440878108446455, 0.7609158263859181, 0.021398541370193903, 0.06298290919938167]"
16468,Pepper,Warning (feat. Stick Figure),51,0.801,0.527,1,-7.191,0,0.1180,0.01920,...,0,0,0,0,0,0,0,0,0,"[1.511675880447574, -0.8555879746040728, -0.02041051791508581, -0.621063961524893, 0.3738271845981228, 0.05918619898462035, 0.11107037679585718, -1.6375937603588562, 0.011027938203398235]"
5125,Kelvyn Boy,Tele,49,0.848,0.743,1,-7.095,0,0.0804,0.36200,...,0,0,0,0,0,0,0,0,0,"[1.793084090541478, 0.18379942646750885, 0.009908841282957543, 0.7023810152118274, -0.055640103902110626, -0.4366089971521044, 0.9062759927415893, -0.8241204656775679, -0.4788146847200311]"
15805,Sheryl Crow,Home,38,0.485,0.409,0,-13.920,1,0.0278,0.24500,...,0,0,0,0,1,0,0,0,0,"[-0.3803452767795279, -1.4234014622265112, -2.145608101702935, 0.250680133420123, -0.6564374277082883, 0.6247158823995809, -0.8252201064950858, 1.4164032781183464, 0.7708677674537697]"


In [3]:
# let's create a column that is all of the normalized audio features we want to use
df_train_audio_normalized = df_train[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].copy()
df_train_audio_normalized = (df_train_audio_normalized-df_train_audio_normalized.mean())/df_train_audio_normalized.std()

df_test_audio_normalized = df_test[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].copy()
df_test_audio_normalized = (df_test_audio_normalized-df_test_audio_normalized.mean())/df_test_audio_normalized.std()

#print(df_train_audio_normalized)

#df_train['normalized_audio_feature_array'] = df_train_audio_normalized.to_numpy()
#df_test['normalized_audio_feature_array'] = df_test_audio_normalized.to_numpy()

df_train['normalized_audio_feature_array'] = df_train_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist()
df_train['normalized_audio_feature_array'] = df_train['normalized_audio_feature_array'].apply(lambda x: np.array(x))


df_test['normalized_audio_feature_array'] = df_test_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist()
df_test['normalized_audio_feature_array'] = df_test['normalized_audio_feature_array'].apply(lambda x: np.array(x))

#display(df_train['normalized_audio_feature_array'])
#df_test['normalized_audio_feature_array'] = df_test_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].to_numpy()

#df_train['normalized_audio_feature_array'] = np.array(df_train_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values)
#df_test['normalized_audio_feature_array'] = np.array(df_test_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values)

#df_train['normalized_audio_feature_array'] = df_train_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist()
#df_test['normalized_audio_feature_array'] = df_test_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist()

#df_train['normalized_audio_feature_array'] = df_train['normalized_audio_feature_array'].apply(lambda x: np.array(x, dtype=np.float64))
#df_test['normalized_audio_feature_array'] = df_test['normalized_audio_feature_array'].apply(lambda x: np.array(x, dtype=np.float64))

#df_train['normalized_audio_feature_array'] = np.array(df_train['normalized_audio_feature_array'])
#df_test['normalized_audio_feature_array'] = np.array(df_test['normalized_audio_feature_array'])

#df_train['normalized_audio_feature_array'] = np.array(df_train_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist())
#df_test['normalized_audio_feature_array'] = np.array(df_test_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist())

In [ ]:
df_train.normalized_audio_feature_array.apply(type)

In [ ]:
df_train['normalized_audio_feature_array']

In [4]:
# get word2vec model
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [6]:
# how big does our embedding matrix need to be
print(len(model.key_to_index.items()))

43981


In [5]:
#construct embedding matrix w/ prebuilt embedding
vocab_dict = model.key_to_index.copy()
embedding_matrix = np.zeros((43982,300))
for word,index in model.key_to_index.items():
    embedding_matrix[index] = model[word]

In [6]:
# bringing in lyric tokenizer function
def text_to_index(text_data,mapping,max_size):
    return_data = []
    for text in text_data:
        new_text = text.lower()
        new_text = text.replace('\n',' ')
        new_text = text.replace('  ',' ')
        new_text = new_text.split()
        mapped_text = []
        for token in new_text:
            try:
                mapped_text.append(mapping[token])
            except:
                mapped_text.append(len(mapping))
        
        if len(mapped_text) > max_size:
            mapped_text = mapped_text[:max_size]
        else:
            while len(mapped_text) < max_size:
                mapped_text.append(len(mapping))
                
        return_data.append(mapped_text)
    
    return return_data

In [7]:
# tokenize train/test lyrics - "get X"
train_tokens_prebuilt = text_to_index(df_train['Lyrics'],vocab_dict,1000)
test_tokens_prebuilt = text_to_index(df_test['Lyrics'],vocab_dict,1000)

In [8]:
# get labels "Y"
train_labels = df_train['Major Genre']
test_labels = df_test['Major Genre']

In [9]:
# create mapper so we can use numeric labels in our networks
mapping = {}
count = 0
for label in train_labels.unique():
    mapping[label] = count
    count = count + 1

In [10]:
# want to keep the functions consistent across notebooks, so defining this so i can use the DAN and WAN models as-is
embedding_matrix_custom = None

DAN Model

In [ ]:
def create_dan_model(retrain_embeddings=False, 
                     max_sequence_length=1000,
                     embedding_matrix=embedding_matrix_custom, 
                     hidden_dim=[100,100,100],
                     dropout_rate=0.3,
                     hidden_layer_activation = 'relu',
                     output_layer_size = 4,
                     output_activation = 'softmax',
                     learning_rate=0.001):
    """
    Construct the DAN model including the compilation and return it. Parametrize it using the arguments.
    retrain_embeddings: bool, indicates whether embeddings are retrainable
    max_sequence_length: Number of token IDs to expect in a given input
    embedding_matrix: initialize embedding layer with embedding matrix, specifying weights
    hidden_dim = number of neurons in hidden layers
    dropout = dropout rate
    output_layer_size = # of neurons in output layer corresponding to # of classes, each neuron predicts P(class K | x)
    output_activation = activation function for output layer
    learning_rate = learning rate for gradient descent for finding model params to optimize loss
    """
    
    #Specify Embedding Layer, including shape, intialize with weights, expected input length, and whether it is trainable
    dan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                  embedding_matrix.shape[1],
                                  weights = [embedding_matrix],
                                  input_length=max_sequence_length,
                                  trainable=retrain_embeddings,
                                   name = 'embedding_layer')
    
    
    #Input Layer, sequence of max_sequence_length tokens
    dan_input_layer = tf.keras.layers.Input(shape=(max_sequence_length,), dtype='int64',name='input')
    #Inputs go into embedding layer, form max_sequence_length x embedding dim matrix
    dan_embeddings = dan_embedding_layer(dan_input_layer)
    #Embeddings are averaged, forming single vector represenation of size embedding matrix
    dan_avg_input_embeddings = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1), name='averaging')(dan_embeddings)
    
    #input into hidden layers
    x = dan_avg_input_embeddings #hidden layer initial input
    count = 1
    for layer in hidden_dim:
        hidden = tf.keras.layers.Dense(layer,activation = hidden_layer_activation,name='hidden_' + str(count))(x)
        dropout = tf.keras.layers.Dropout(dropout_rate,name='dropout_' + str(count))(hidden)
        count = count + 1
        x = dropout
        
    #dan_hidden_out_1 = tf.keras.layers.Dense(hidden_dim, activation='relu', name='hidden_1')(dan_avg_input_embeddings)
    #dan_hidden_out_1 = tf.keras.layers.Dropout(dropout)(dan_hidden_out_1)
    dan_classification = tf.keras.layers.Dense(output_layer_size, activation='softmax', name='dan_classification')(x)
    dan_model = tf.keras.models.Model(inputs=dan_input_layer, outputs=[dan_classification])
    dan_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-07,
                                                amsgrad=False,
                                                name='Adam'),
                 metrics='accuracy')
    
    print(dan_model.summary())

    return dan_model

In [ ]:
dan_model_sorted = create_dan_model(embedding_matrix = embedding_matrix, output_layer_size = 7)
dan_sorted_history = dan_model_sorted.fit(np.array(train_tokens_prebuilt),
                        np.array(train_labels.map(mapping)),
                        validation_data=(np.array(test_tokens_prebuilt), np.array(test_labels.map(mapping))),
                        batch_size=8,
                        epochs=10,
                        shuffle=True,
                        use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

40% validation accuracy after 10 epochs - so not very good!  but the model is learning a little, at least

Lets see if a WAN will perform any better

In [ ]:
def create_wan_model(retrain_embeddings=False, 
                     max_sequence_length=1000,
                     embedding_matrix=embedding_matrix_custom,
                     num_attention = 1,
                     hidden_dim=[100,100,100],
                     dropout_rate=0.3,
                     hidden_layer_activation = 'relu',
                     output_layer_size = 4,
                     output_activation = 'softmax',
                     learning_rate=0.001):
    """
    Construct the WAN model including the compilation and return it. Parametrize it using the arguments.
    retrain_embeddings: bool, indicates whether embeddings are retrainable
    max_sequence_length: Number of token IDs to expect in a given input
    embedding_matrix: initialize embedding layer with embedding matrix, specifying weights
    num_attention = number of parallel attention computations that learn how to balance embeddings into a single
    vector representation, final attention layer weights prior attention based representations
    hidden_dim = number of neurons in hidden layers
    dropout = dropout rate
    output_layer_size = # of neurons in output layer corresponding to # of classes, each neuron predicts P(class K | x)
    output_activation = activation function for output layer
    learning_rate = learning rate for gradient descent for finding model params to optimize loss
    """
    
    #Specify Embedding Layer, including shape, intialize with weights, expected input length, and whether it is trainable
    wan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                  embedding_matrix.shape[1],
                                  weights = [embedding_matrix],
                                  input_length=max_sequence_length,
                                  trainable=retrain_embeddings,
                                   name = 'embedding_layer')
    
    
    #Input Layer, sequence of max_sequence_length tokens
    wan_input_layer = tf.keras.layers.Input(shape=(max_sequence_length,), dtype='int64',name='input')
    #Inputs go into embedding layer, form max_sequence_length x embedding dim matrix
    wan_embeddings = wan_embedding_layer(wan_input_layer)
    
    #Create attention based single vector representations of words according to alternative query vectors
    attention_embeddings = []
    for num in range(num_attention):
        #Apply Query Vector to words in embeddings, returning a max_sequence_length x 1 tensor
        l1_query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query' + str(num+1))(wan_embeddings)
        #reshape to 1 x max_sequence_length
        l1_reshape_query = tf.keras.layers.Reshape((1,max_sequence_length))(l1_query)
        #Softmax over query * key (words) to obtain weights
        l1_weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                            name='attention_weights' + str(num+1))(l1_reshape_query)
        #weight embeddings according to weights
        l1_attention = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((wan_embeddings,l1_weights)))
        attention_embeddings.append(l1_attention)
    
    concat_attention = tf.keras.layers.Concatenate()(attention_embeddings)
    concat_attention = tf.keras.layers.Reshape((num_attention,embedding_matrix.shape[1]))(concat_attention)
    
    #Apply Query Vector to attention based representations, returning a num_attention x 1 tensor
    wan_query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query')(concat_attention)
    #reshape to 1 x num_attention
    reshaped_query = tf.keras.layers.Reshape((1,num_attention))(wan_query)
    #Softmax over query * key (words) to obtain weights
    wan_weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                        name='attention_weights')(reshaped_query)
    #weight attention embeddings according to weights, learning how to balance attention based vector representations 
    #from prior layer
    wan_attention = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((concat_attention,wan_weights)))
    
    #input into hidden layers
    x = wan_attention #hidden layer initial input
    count = 1
    for layer in hidden_dim:
        hidden = tf.keras.layers.Dense(layer,activation = hidden_layer_activation,name='hidden_' + str(count))(x)
        dropout = tf.keras.layers.Dropout(dropout_rate,name='dropout_' + str(count))(hidden)
        count = count + 1
        x = dropout
        
    #wan_hidden_out_1 = tf.keras.layers.Dense(hidden_dim, activation='relu', name='hidden_1')(wan_avg_input_embeddings)
    #wan_hidden_out_1 = tf.keras.layers.Dropout(dropout)(wan_hidden_out_1)
    wan_classification = tf.keras.layers.Dense(output_layer_size, activation='softmax', name='wan_classification')(x)
    wan_model = tf.keras.models.Model(inputs=wan_input_layer, outputs=[wan_classification])
    wan_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-07,
                                                amsgrad=False,
                                                name='Adam'),
                 metrics='accuracy')
    
    print(wan_model.summary())

    return wan_model

In [ ]:
wan_model_sorted = create_wan_model(embedding_matrix=embedding_matrix, output_layer_size = 7,
                                   num_attention=1)
wan_sorted_history = wan_model_sorted.fit(np.array(train_tokens_prebuilt),
                        np.array(train_labels.map(mapping)),
                        validation_data=(np.array(test_tokens_prebuilt), np.array(test_labels.map(mapping))),
                        batch_size=8,
                        epochs=10,
                        shuffle=True,
                        use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

41%...so not much better

Both of those models used prebuilt embeddings, what happens if we use custom ones?

In [13]:
# initialize vectorizer with preprocessing
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.replace('  ',' ')
    return text

vectorizer = CountVectorizer(preprocessor=preprocess_text)

DO NOT RUN BELOW CELL - it will crash the kernel.  need to figure out a workaround

In [ ]:
#Vectorize Train Lyrics
#train_lyrics = vectorizer.fit_transform(df_train['Lyrics'])
#train_lyrics = pd.DataFrame(train_lyrics.todense(),columns = vectorizer.get_feature_names())
#train_lyrics_token_count = train_lyrics.sum(axis=1)
#train_lyrics = train_lyrics/np.array(train_lyrics_token_count.repeat(len(train_lyrics.columns))).reshape(train_lyrics.shape)

#Vectorize Test Lyrics
#test_lyrics = vectorizer.transform(df_test['Lyrics'])
#test_lyrics = pd.DataFrame(test_lyrics.todense(),columns = vectorizer.get_feature_names())
#test_lyrics_token_count = test_lyrics.sum(axis=1)
#test_lyrics = test_lyrics/np.array(test_lyrics_token_count.repeat(len(test_lyrics.columns))).reshape(test_lyrics.shape)



In [ ]:
#If we can get vectorizer working, use this to make custom embedding matrix

#vocab_dict_custom = {}
#count = 0
#for word in vectorizer.get_feature_names():
#    vocab_dict_custom[word] = count
#    count = count + 1
#embedding_matrix_custom = np.random.random((len(vectorizer.get_feature_names()) + 1,300))
#embedding_matrix_custom[-1] = 0

lets look at the audio features we have too

In [14]:
# audio features should help too: lets see what results we get from
# a standard feed-forward network
# note: audio features have been normalized

train_normalized_audio_features = np.array(df_train['normalized_audio_feature_array'])
test_normalized_audio_features = np.array(df_test['normalized_audio_feature_array'])

train_normalized_audio_features_1 = df_train['normalized_audio_feature_array']
test_normalized_audio_features_1 = df_test['normalized_audio_feature_array']

model = keras.Sequential([
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(7,activation='softmax')
])

#Compile the model, specifying loss function, optimizer, and performance metric
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
             optimizer = keras.optimizers.Adam(learning_rate=0.01),
             metrics=['accuracy'],
             )

model.fit(x = df_train_audio_normalized,y = train_labels.map(mapping),batch_size=8,epochs=10,
         validation_data = (df_test_audio_normalized ,test_labels.map(mapping)),
         use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

Epoch 1/10
 648/2063 [========>.....................] - ETA: 7s - loss: 1.5232 - accuracy: 0.4242

KeyboardInterrupt: 

In [15]:
print(train_normalized_audio_features)

[array([ 0.0866726 ,  0.55432179,  0.1220273 , -0.67410986, -0.41543318,
        -0.44009997,  0.27353174, -0.04503026, -0.63357693])
 array([ 1.76913446,  0.39552649,  0.026016  , -0.33189818,  0.53373522,
        -0.4389349 ,  0.19657636, -0.45322353, -1.48604061])
 array([ 0.16450892,  0.22229526,  0.59071407, -0.64693059, -0.66671723,
        -0.44061023, -0.48747148, -0.72341021, -0.48175945])
 ...
 array([ 1.79308409,  0.18379943,  0.00990884,  0.70238102, -0.0556401 ,
        -0.436609  ,  0.90627599, -0.82412047, -0.47881468])
 array([-0.38034528, -1.42340146, -2.1456081 ,  0.25068013, -0.65643743,
         0.62471588, -0.82522011,  1.41640328,  0.77086777])
 array([ 0.25432005, -0.67754476, -1.23286906, -0.64654452, -0.48624959,
         3.14196079, -0.38058901,  0.77423566,  0.79733172])            ]


In [19]:
# now lets combine audio + lyrics and see what results look like
#train_tokens_prebuilt = np.array(train_tokens_prebuilt)
#test_tokens_prebuilt = np.array(test_tokens_prebuilt)
df_train_audio_normalized['Lyrics'] = train_tokens_prebuilt
df_test_audio_normalized['Lyrics'] = test_tokens_prebuilt

In [20]:
#df_train_audio_normalized
print(df_train_audio_normalized)

       danceability    energy  loudness  acousticness  speechiness  \
16303      0.086673  0.554322  0.122027     -0.674110    -0.415433   
8721       1.769134  0.395526  0.026016     -0.331898     0.533735   
11930      0.164509  0.222295  0.590714     -0.646931    -0.666717   
7945       0.865036 -0.441758 -0.958100     -0.387106     2.007174   
15504     -0.530030 -0.499502  0.518074     -0.549641    -0.626740   
...             ...       ...       ...           ...          ...   
12579      1.457789  0.809357 -0.472358     -0.679669     2.783870   
16468      1.511676 -0.855588 -0.020411     -0.621064     0.373827   
5125       1.793084  0.183799  0.009909      0.702381    -0.055640   
15805     -0.380345 -1.423401 -2.145608      0.250680    -0.656437   
2952       0.254320 -0.677545 -1.232869     -0.646545    -0.486250   

       instrumentalness   valence     tempo  duration_ms  \
16303         -0.440100  0.273532 -0.045030    -0.633577   
8721          -0.438935  0.196576 -0.45

In [21]:
# first try basic FFN
model = keras.Sequential([
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(7,activation='softmax')
])

#Compile the model, specifying loss function, optimizer, and performance metric
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
             optimizer = keras.optimizers.Adam(learning_rate=0.01),
             metrics=['accuracy'],
             )

model.fit(x = df_train_audio_normalized,y = train_labels.map(mapping),batch_size=8,epochs=10,
         validation_data = (df_test_audio_normalized ,test_labels.map(mapping)),
         use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [22]:
df_train

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,Sub-Genre: southern hip hop,Sub-Genre: nu metal,Sub-Genre: israeli mediterranean,Sub-Genre: thrash metal,Sub-Genre: pop rock,Sub-Genre: chicago blues,Sub-Genre: indie pop,Sub-Genre: classic rock,Sub-Genre: hardcore hip hop,normalized_audio_feature_array
16303,Escape the Fate,Unbreakable,47,0.563,0.820,1,-6.740,0,0.0489,0.00546,...,0,0,0,0,0,0,0,0,0,"[0.08667260380184501, 0.5543217870346929, 0.12..."
8721,COUCOU CHLOE,NOBODY,46,0.844,0.787,11,-7.044,1,0.1320,0.09410,...,0,0,0,0,0,0,0,0,0,"[1.769134455639869, 0.3955264896487572, 0.0260..."
11930,beabadoobee,Worth It,51,0.576,0.751,2,-5.256,1,0.0269,0.01250,...,0,0,0,0,0,0,1,0,0,"[0.16450891723207395, 0.22229525613682674, 0.5..."
7945,Andrew Broder,Bloodrush,39,0.693,0.613,11,-10.160,0,0.2610,0.07980,...,0,0,0,0,0,0,0,0,0,"[0.8650357381041337, -0.4417578056589061, -0.9..."
15504,Greta Van Fleet,Age Of Man,56,0.460,0.601,6,-5.486,0,0.0304,0.03770,...,0,0,0,0,0,0,0,0,0,"[-0.5300304949145832, -0.49950155016288295, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12579,ABSOLUTE.,Sage comme une image - Good as Gold,44,0.792,0.873,6,-8.622,0,0.3290,0.00402,...,0,0,0,0,0,0,0,0,0,"[1.457789201918954, 0.8093566585939238, -0.472..."
16468,Pepper,Warning (feat. Stick Figure),51,0.801,0.527,1,-7.191,0,0.1180,0.01920,...,0,0,0,0,0,0,0,0,0,"[1.511675880447574, -0.8555879746040728, -0.02..."
5125,Kelvyn Boy,Tele,49,0.848,0.743,1,-7.095,0,0.0804,0.36200,...,0,0,0,0,0,0,0,0,0,"[1.793084090541478, 0.18379942646750885, 0.009..."
15805,Sheryl Crow,Home,38,0.485,0.409,0,-13.920,1,0.0278,0.24500,...,0,0,0,0,1,0,0,0,0,"[-0.3803452767795279, -1.4234014622265112, -2...."


In [27]:
print(type(train_tokens_prebuilt[0]))

<class 'list'>


In [11]:
train_audio_features_to_add = df_train_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist()
test_audio_features_to_add = df_test_audio_normalized[['danceability', 'energy', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'valence', 'tempo','duration_ms']].values.tolist()

In [12]:
# append features to tokens

for token_list, feature_list in zip(train_tokens_prebuilt, train_audio_features_to_add):
    for feature in feature_list:
        token_list.append(feature)


for token_list, feature_list in zip(test_tokens_prebuilt, test_audio_features_to_add):
    for feature in feature_list:
        token_list.append(feature)



In [13]:
# first try basic FFN
model = keras.Sequential([
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(7,activation='softmax')
])

#Compile the model, specifying loss function, optimizer, and performance metric
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
             optimizer = keras.optimizers.Adam(learning_rate=0.01),
             metrics=['accuracy'],
             )

model.fit(x = np.array(train_tokens_prebuilt),y = train_labels.map(mapping),batch_size=8,epochs=10,
         validation_data = (np.array(test_tokens_prebuilt), test_labels.map(mapping)),
         use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

Epoch 1/10
2063/2063 [==============================] - 8s 4ms/step - loss: 2410.3469 - accuracy: 0.3086 - val_loss: 2.5497 - val_accuracy: 0.3012
Epoch 2/10
2063/2063 [==============================] - 9s 4ms/step - loss: 1.8301 - accuracy: 0.3147 - val_loss: 2.6634 - val_accuracy: 0.3012
Epoch 3/10
2063/2063 [==============================] - 9s 4ms/step - loss: 1.8307 - accuracy: 0.3147 - val_loss: 2.6675 - val_accuracy: 0.3012
Epoch 4/10
2063/2063 [==============================] - 8s 4ms/step - loss: 1.8306 - accuracy: 0.3147 - val_loss: 2.6660 - val_accuracy: 0.3012
Epoch 5/10
2063/2063 [==============================] - 7s 4ms/step - loss: 1.8303 - accuracy: 0.3147 - val_loss: 2.6672 - val_accuracy: 0.3012
Epoch 6/10
2063/2063 [==============================] - 8s 4ms/step - loss: 1.8300 - accuracy: 0.3147 - val_loss: 2.6643 - val_accuracy: 0.3012
Epoch 7/10
2063/2063 [==============================] - 9s 4ms/step - loss: 1.8301 - accuracy: 0.3147 - val_loss: 2.6702 - val_accura

In [14]:
unk, total = 0,0
for token_list in train_tokens_prebuilt:
    for token in token_list:
        if token == 43981:
            unk += 1
        total += 1
print(unk/total)

0.7537845451542181


In [22]:
print(train_tokens_prebuilt[1][300:])
print(len())

[11726, 16915, 943, 30943, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 

In [36]:
df_train['Lyrics']

16303                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [45]:
def avg_text_length(text_data):
    return_data = []
    for text in text_data:
        new_text = text.lower()
        new_text = text.replace('\n',' ')
        new_text = text.replace('  ',' ')
        new_text = new_text.split()
        return_data.append(new_text)
    return(return_data)
        


In [46]:
new_lyrics = avg_text_length(df_train['Lyrics'])

In [56]:
total_len = 0
iter = 0
old_len = 0


for lyric in new_lyrics:
    total_len += len(lyric)
    len_lyric = len(lyric)
    iter += 1
print(total_len/len(new_lyrics))
print(len(new_lyrics))

print(new_lyrics[14001])
print(len(new_lyrics[14001]))

2020.9347878787878
16500
['Movin’', 'On', 'Up', 'Lyrics[Verse', '1]', 'I', 'was', 'blind,', 'now', 'I', 'can', 'see', 'You', 'made', 'a', 'believer', 'out', 'of', 'me', 'I', 'was', 'blind,', 'now', 'I', 'can', 'see', 'You', 'made', 'a', 'believer', 'out', 'of', 'me', '[Chorus]', "I'm", "movin'", 'on', 'up', 'now', 'Getting', 'out', 'of', 'the', 'darkness', 'My', 'light', 'shines', 'on', 'My', 'light', 'shines', 'on', 'My', 'light', 'shines', 'on', '[Verse', '2]', 'I', 'was', 'lost,', 'now', "I'm", 'found', 'I', 'believe', 'in', 'you,', "I've", 'got', 'no', 'bounds', 'I', 'was', 'lost,', 'now', "I'm", 'found', 'I', 'believe', 'in', 'you,', "I've", 'got', 'no', 'bounds', '[Chorus]', "I'm", "movin'", 'on', 'up', 'now', 'Getting', 'out', 'of', 'the', 'darkness', 'My', 'light', 'shines', 'on', 'My', 'light', 'shines', 'on', 'My', 'light', 'shines', 'on', '(My', 'light', 'shines', 'on)', 'My', 'light', 'shines', 'on', '(My', 'light', 'shines', 'on)', 'My', 'light', 'shines', 'on', '[Bridge]'

In [87]:
#new_lyrics.sort(key = len, reverse = True)
#print(new_lyrics[1][0:100])

#for lyric in new_lyrics[0:1500]:
 #   print(len(lyric))
    
print(new_lyrics[13556])

['An', 'Iteration', 'Lyrics[Verse', '1]', 'I', 'fell', 'for', 'some', 'Pseudo-sophisticated', 'Poet', 'laureate-posing', 'Young', 'white', 'savior', 'He', 'sang', 'to', 'me', 'A', 'blue', 'collar', 'emulation', 'An', 'accent', 'so', 'affected', 'So', 'midwestern', '[Pre-Chorus', '1]', '"Shots', 'in', 'the', 'dark', 'Kalashnokov', "You'll", 'take', 'her', 'eye', 'out"', 'Too', 'cool', 'to', 'be', 'trite', 'His', 'cunning', 'remarks', 'And', 'then', 'he', 'did', 'it', 'again,', 'did', 'it', 'again,', 'did', 'it', 'again', 'Did', 'it', 'again,', 'did', 'it', 'again,', 'did', 'it', 'again', '[Chorus]', 'An', 'iteration', 'An', 'iteration', 'An', 'iteration', '[Verse', '2]', 'He', 'said', 'to', 'me', '"You\'ve', 'got', 'your', 'own', 'reflection"', "It's", 'extra', 'underwhelming', 'It', 'lets', 'me', 'down', '[Pre-Chorus', '2]', "Stroh's", 'from', 'the', 'tap,', 'working', 'class', 'porn', 'And', "daddy's", 'money', 'Too', 'cool', 'to', 'excite', "He's", 'up', 'there', 'so', 'bored', 'And'

In [51]:
print(train_tokens_prebuilt[11200])

[43981, 43981, 43981, 1465, 43981, 34782, 35357, 43981, 36633, 33259, 9511, 43981, 19666, 6, 7347, 30496, 10161, 3073, 11992, 43981, 22793, 43981, 22793, 23973, 43700, 28117, 27498, 43981, 1465, 43981, 39418, 21386, 588, 43981, 4110, 28117, 43981, 14193, 43981, 3231, 28117, 10490, 43981, 27452, 43981, 16915, 36945, 43981, 16915, 14193, 43981, 43981, 36945, 38967, 30212, 28117, 43981, 14193, 43981, 12728, 19334, 40473, 41394, 9059, 36645, 14193, 43981, 12728, 32865, 3821, 43981, 43981, 43981, 37663, 19870, 12369, 43981, 16834, 19870, 7033, 6071, 37060, 25687, 43981, 16834, 19870, 7033, 6071, 43981, 43981, 1465, 43981, 43981, 37060, 17131, 43981, 43981, 23844, 12292, 43981, 24652, 12139, 43817, 36633, 12139, 24344, 12917, 43069, 20514, 20354, 16915, 36945, 43981, 37008, 43817, 40367, 18991, 14446, 43981, 12139, 12289, 43981, 6071, 20514, 19334, 34707, 43981, 1465, 43981, 39418, 21386, 588, 43981, 4110, 28117, 43981, 14193, 43981, 3231, 28117, 10490, 43981, 27452, 43981, 16915, 36945, 439

In [35]:
pd.options.display.max_colwidth = 5000

In [40]:
df_train['Lyrics'] = d

AttributeError: 'Series' object has no attribute 'split'

In [38]:
mean_length = df_train['Lyrics'].apply(len).mean()

In [39]:
print(mean_length)

11293.51103030303


In [73]:
s = df_train['Lyrics'].str.len().sort_values().index

In [76]:
print(s)

Int64Index([14097,  2299,  3159,  3189,  3365, 15600, 18570,  2474,  1366,
             2968,
            ...
             1896, 13649,  2574,  9360,  5683,   854,  1299,  2947,  3709,
            10720],
           dtype='int64', length=16500)


In [77]:
df_train_reindexed = df_train.reindex(s)

In [78]:
df_train_reindexed_subset = df_train_reindexed.tail(100)

In [80]:
df_train_reindexed_subset.to_excel('reindex_test.xlsx')

In [94]:
df_train_reindexed_subset_2 = df_train_reindexed[df_train_reindexed['Artist Name'] == 'AJ Tracey']

In [95]:
df_train_reindexed_subset_3 = df_train_reindexed[df_train_reindexed['Artist Name'] == 'Kanye West']

In [96]:
df_train_reindexed_subset_2.to_excel('reindex_test_aj.xlsx')
df_train_reindexed_subset_3.to_excel('reindex_test_kanye.xlsx')